## 🎯 Deep Hedging Section 5.2 再現ノートブック（公式コードベース）

In [30]:
import sys
import os
sys.path.append(".")

from deephedging.trainer import train
from deephedging.gym import VanillaDeepHedgingGym
from deephedging.world import SimpleWorld_Spot_ATM
from cdxbasics.config import Config
import matplotlib.pyplot as plt
import numpy as np
import yaml
from IPython.display import display, Markdown


In [32]:
from deephedging.trainer import wrap_config
from cdxbasics.config import Config
import yaml

# 1. YAML読み込み
with open("config.yaml", "r", encoding="utf-8") as f:
    config_dict = yaml.safe_load(f)

# 2. 再帰的に Config オブジェクトに変換
def to_config(d):
    if isinstance(d, dict):
        return Config({k: to_config(v) for k, v in d.items()})
    return d

config = to_config(config_dict)

# 3. 必要な箇所だけ wrap_config で CdxConfigWrapper に変換（これが最重要）
config_wrapped = wrap_config(config)
world_wrapped = wrap_config(config_dict.get("world", {}))

# 4. World, Gym, Trainer 実行
world = SimpleWorld_Spot_ATM(world_wrapped)
val_world = world.clone(samples=world.nSamples // 2)
gym = VanillaDeepHedgingGym(config_wrapped)

train(gym=gym, world=world, val_world=val_world, config=config_wrapped)






TypeError: 'NoneType' object is not callable

In [ ]:
print("gym section type:", type(config_dict["gym"]))
print("after to_config:", type(config.gym))

gym section type: <class 'dict'>
after to_config: <class 'cdxbasics.config.Config'>


In [ ]:
world = SimpleWorld_Spot_ATM(config.world)
val_world = world.clone(samples=world.nSamples // 2)
gym = VanillaDeepHedgingGym(config.gym)

train(gym=gym, world=world, val_world=val_world, config=config)


TypeError: 'TrackedDict' object is not callable

In [ ]:
# ■Indifference Price の算出
r = gym(world.tf_data)
pi_minus_z = np.mean(r["utility"])
pi_0 = np.mean(r["utility0"])
indiff_price = pi_minus_z - pi_0

print(f"📌 Indifference Price: π(-Z) − π(0) = {indiff_price:.4f}")


In [ ]:
plt.plot(gym.history["loss"])
plt.title("CVaR Loss during Training")
plt.xlabel("Epoch")
plt.ylabel("CVaR Loss")
plt.grid(True)
plt.show()


In [ ]:
# ■デルタ比較グラフ（Black-Scholes vs Deep Hedging）
import deephedging.plot_bs_hedge as pb

# ⚠️ 論文5.2ではHestonデルタヘッジが比較対象。
# この関数はBSデルタを使っているため、参考程度にどうぞ。
pb.plot(gym, world)